# Analyza liekov podla skupiny ATC7 
* pozn. Doplnit info o liekoch, popis!!, upravit grafy a farby

Analýzu sme robili na dátach z roku 2019, keďže je to posledný rok, z ktorého sme mali údaje za všetky štvrťroky a zároveň v tomto roku na Slovensku ešte nebola pandémia.
Otázky:
* Ktoré lieky s danou účinnou látkou sa predávajú viac než ostatné? Závisí predaj od krajiny výrobcu?

(mozno fajn vo vsetkych grafoch krajiny rovnakou farbou)
 1409 roznych ucinnych latok

In [9]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px
%run tables.py

# Tabulka, s ktorou pracujeme (rok 2019) s vybranymi udajmi

In [10]:
table = import_table('detailed', 2019)
table = table.rename(columns={'Počet balení': 'Počet_balení'})

In [11]:

table2019=table.loc[:,['ATC7_KOD','ATC7_popis','LIEK_KOD','LIEK_popis', 'STAT_popis', 'VYROBCA_popis', 'Počet_balení','Úhrada_PAC']]
table2019 = table2019.query('ATC7_KOD != "(prazdne)"').query( 'ATC7_popis != "neznámy"')
table2019sum = table2019.groupby(['ATC7_KOD','ATC7_popis','LIEK_KOD','LIEK_popis', 'STAT_popis','VYROBCA_popis']).sum().reset_index()
display(table2019sum.head())
display(table2019sum.tail())


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,A01AA03,Olaflur,C40630,ELMEX FLUID sol dnt 1x50 ml,Nemecko,GABA GmbH,40.0,135.08
1,A01AA30,"Profylaxia zubného kazu, kombinácie",C40631,ELMEX gelee gel dnt 1x25 g,Nemecko,CP GABA GmbH,43832.5,313077.14
2,A01AA30,"Profylaxia zubného kazu, kombinácie",C85867,ELMEX gelee gel dnt 1x215 g,Nemecko,CP GABA GmbH,29.0,171.85
3,A01AB03,Chlorhexidín,C14106,CORSODYL sol ora 1x600 ml,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,72.0,254.33
4,A01AB03,Chlorhexidín,C34296,PROTEFIX ANTISEPT sol der 200 ml,Nemecko,Queisser Pharma GmbH & Co.,6.0,27.82


,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
9813,V12,Homeopatiká,C82842,GINKGO BILOBA plv 15 g ch2-ch200,Francúzsko,Laboratoires BOIRON,309.0,882.18
9814,V12,Homeopatiká,C96701,ARNICA MONTANA ung 20 g ch2-ch30,Francúzsko,Laboratoires BOIRON,15.0,52.29
9815,V12,Homeopatiká,C97944,QUIETUDE tbl 60,Francúzsko,Laboratoires BOIRON,156.0,1030.21
9816,V12,Homeopatiká,C99176,BELLADONNA gra 4 g ch3-ch30,Francúzsko,Laboratoires BOIRON,183.0,631.73
9817,V12,Homeopatiká,C99177,ARNICA MONTANA gra 4 g ch2-ch30,Francúzsko,Laboratoires BOIRON,700.0,2375.06


# Tabulka: 15 najpredavanejsich ucinnych latok

In [12]:
atc7 = table2019.loc[:,['ATC7_KOD', 'ATC7_popis','Počet_balení']].groupby(['ATC7_KOD', 'ATC7_popis']).sum().sort_values(by='Počet_balení', ascending=False).reset_index().head(15)
display(atc7)

,ATC7_KOD,ATC7_popis,Počet_balení
0,V09FX01,Technécium - (,9618479.510
1,B05BB01,Elektrolyty,5331686.852
2,N02BE01,Paracetamol,3297020.107
3,M01AE01,Ibuprofén,2795946.717
4,C07AB07,Bisoprolol,2607372.542
5,C08CA01,Amlodipín,2119340.883
6,N05BA12,Alprazolam,2045121.730
7,M01AB05,Diklofenak,2013726.518
8,N02BB02,"Metamizol, sodná soľ",1946250.707
9,C07AB02,Metoprolol,1873602.519


In [37]:
fig = px.bar(atc7.sort_values(by='Počet_balení'), x='Počet_balení', y='ATC7_popis', orientation='h')
fig.show()

## Ukazky udajov jednotlivych tabuliek (podla poctu baleni)

In [13]:
nazvy = []
for n in atc7['ATC7_KOD']:
    if n not in nazvy:
        nazvy.append(n)

top20tab= {}
for i in range(20):
    top20tab[nazvy[i]] = table2019sum.query(f'ATC7_KOD == "{nazvy[i]}"').sort_values(by='Počet_balení', ascending= False).reset_index()
    display(top20tab[nazvy[i]].head())

,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,9547,V09FX01,Technécium - (,C91102,"Drytec gen rnk 2,5 GBq",Holandsko,GE Healthcare B.V.,6688913.50,0.0
1,9546,V09FX01,Technécium - (,C15457,"Drytec gen rnk 10,0 GBq",Holandsko,GE Healthcare B.V.,1469870.00,0.0
2,9541,V09FX01,Technécium - (,C15308,"Ultra-TechneKow FM (99mTC-generátor) inj 10,63...",Holandsko,Curium Netherlands B.V.,991979.00,0.0
3,9545,V09FX01,Technécium - (,C15456,"Drytec gen rnk 9,0 GBq",Holandsko,GE Healthcare B.V.,467689.01,0.0
4,9540,V09FX01,Technécium - (,C15307,"Ultra-TechneKow FM (99mTC-generátor) inj 8,50 ...",Holandsko,Curium Netherlands B.V.,14.00,0.0


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,1450,B05BB01,Elektrolyty,C37743,"0,9% SODIUM CHL.IN W.F.I.FRESENIUS sol inf 1x1...",Česko,Fresenius Kabi s.r.o.,1115466.27,1758.64
1,1432,B05BB01,Elektrolyty,C33593,"0,9% SODIUM ChL.IN W.F.I.FRESENIUS sol inf 1x2...",Česko,Fresenius Kabi s.r.o.,999336.36,4893.52
2,1431,B05BB01,Elektrolyty,C33590,"0,9% SODIUM ChL.IN W.F.I.FRESENIUS sol inf 1x5...",Česko,Fresenius Kabi s.r.o.,789546.85,1375.02
3,1505,B05BB01,Elektrolyty,C97748,"SODIUM CHLORIDE 0,9%W/V I.V. INF inf 1x100 ml ...",Taliansko,Bieffe Medital S.p.A.,418659.15,5287.02
4,1506,B05BB01,Elektrolyty,C97749,"SODIUM CHLORIDE 0,9%W/V I.V. INF inf 1x250 ml ...",Taliansko,Bieffe Medital S.p.A.,381477.64,18777.30


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,6299,N02BE01,Paracetamol,C86052,PARALEN 500 tbl 24x500 mg (blis.Al/PVC),Slovensko,sanofi-aventis Slovakia s.r.o.,2300126.084,3826254.68
1,6298,N02BE01,Paracetamol,C86051,PARALEN 500 tbl 12x500 mg (blis.Al/PVC),Slovensko,sanofi-aventis Slovakia s.r.o.,269140.730,279950.53
2,6287,N02BE01,Paracetamol,C27613,PARALEN 100 sup 5x100 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,139481.200,117364.89
3,6290,N02BE01,Paracetamol,C57476,Panadol Paracetamolum 500 mg tbl flm 24x500 mg...,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,113876.250,194443.83
4,6281,N02BE01,Paracetamol,C26627,PARALEN 125 tbl 20x125 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,111906.328,81286.36


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,5604,M01AE01,Ibuprofén,C35458,IBALGIN 400 tbl flm 24x400 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,596621.970,1885611.99
1,5640,M01AE01,Ibuprofén,C7880A,Ibalgin 400 tbl flm 48x400 mg (blis. PVC/Al),Slovensko,sanofi-aventis Slovakia s.r.o.,293384.731,1432962.47
2,5651,M01AE01,Ibuprofén,C97299,BRUFEN 400 tbl flm 30x400 mg,Francúzsko,Mylan Laboratories SAS,232798.061,868313.80
3,5618,M01AE01,Ibuprofén,C49851,MIG-400 tbl flm 30x400 mg,Nemecko,Berlin-Chemie AG,205849.323,706016.17
4,5610,M01AE01,Ibuprofén,C45451,NUROFEN pre deti Jahoda sus por 1x100 ml (PET ...,Česko,"Reckitt Benckiser (Czech Republic), spol. s r.o.",160812.710,747388.09


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,2001,C07AB07,Bisoprolol,C94164,CONCOR 5 tbl flm 30x5 mg,Slovensko,Merck spol. s.r.o.,861847.12,2312824.20
1,1997,C07AB07,Bisoprolol,C90028,CORONAL 5 tbl flm 30x5 mg (PVC/PE/PVDC/Alu blis.),Slovensko,ZNT-SK-2?,439825.33,198924.15
2,1974,C07AB07,Bisoprolol,C33446,BISOPROLOL-ratiopharm 5 mg tbl 30x5 mg,Nemecko,ratiopharm GmbH,361429.00,71700.98
3,1971,C07AB07,Bisoprolol,C33427,"CONCOR COR 2,5 mg tbl flm 30x2,5 mg",Slovensko,Merck spol. s.r.o.,174083.00,439473.10
4,2004,C07AB07,Bisoprolol,C97761,BISOMERCK 5 tbl flm 30x5 mg,Slovensko,Merck spol. s.r.o.,116380.50,156601.14


...

,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,6367,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C65951,TheraFlu prechladnutie a chrípka plo por 1x14 ...,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,422718.171,3954421.86
1,6334,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C30229,PARALEN PLUS tbl flm 24,Slovensko,sanofi-aventis Slovakia s.r.o.,100459.030,589486.22
2,6342,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C33813,THERAFLU PRECHLADNUTIE A CHRÍPKA plo por 1x10 ...,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,81452.991,671984.16
3,6347,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C36516,PARALEN HOT DRINK plu por 1x12 vreciek,Slovensko,sanofi-aventis Slovakia s.r.o.,65515.562,481851.44
4,6311,N02BE51,"Paracetamol, kombinácie okrem psycholeptík",C0381A,PARALEN GRIP chrípka a bolesť tbl flm 1x24 (bl...,Slovensko,sanofi-aventis Slovakia s.r.o.,61588.890,362517.93


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,5780,M02AA15,Diklofenak,C11849,"Voltaren Forte 2,32 % gél gel der 1x100 g (tub...",Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,189743.17,2256703.40
1,5781,M02AA15,Diklofenak,C11851,"Voltaren Forte 2,32 % gél gel der 1x150 g (tub...",Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,136412.90,2083185.45
2,5785,M02AA15,Diklofenak,C18515,FLECTOR EP TISSUGEL emp tdm 5,Slovensko,IBSA Slovakia s.r.o.,136175.20,1201954.19
3,5784,M02AA15,Diklofenak,C18514,FLECTOR EP gel 1x100 g,Slovensko,IBSA Slovakia s.r.o.,112500.25,884399.45
4,5808,M02AA15,Diklofenak,C84453,DICLOBENE gel 1x100 g,Holandsko,TEV-NL-4?,94815.70,515849.66


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,782,A11GA01,Kyselina askorbová (vitamín C),C78278,CELASKON 250 tbl 100x250 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,396438.278,1796114.28
1,781,A11GA01,Kyselina askorbová (vitamín C),C78277,CELASKON 250 tbl 30x250 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,250880.500,389292.20
2,778,A11GA01,Kyselina askorbová (vitamín C),C42420,CELASKON 500 mg červený pomaranč tbl eff 30x50...,Slovensko,sanofi-aventis Slovakia s.r.o.,106336.777,602810.11
3,777,A11GA01,Kyselina askorbová (vitamín C),C35913,CELASKON LONG EFFECT cps pld 60x500 mg,Slovensko,sanofi-aventis Slovakia s.r.o.,79906.524,654347.40
4,771,A11GA01,Kyselina askorbová (vitamín C),C24836,ACIDUM ASCORBICUM BIOTIKA sol inj 5x5 ml/500 mg,Česko,BB Pharma a.s.,79579.400,42800.62


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,138,A02BC02,Pantoprazol,C43345,Nolpaza 20 mg tbl ent 30x20 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",184810.510,255770.78
1,139,A02BC02,Pantoprazol,C43349,Nolpaza 20 mg tbl ent 100x20 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",144399.293,632243.24
2,142,A02BC02,Pantoprazol,C43362,Nolpaza 40 mg tbl ent 100x40 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",117848.280,648739.25
3,141,A02BC02,Pantoprazol,C43358,Nolpaza 40 mg tbl ent 30x40 mg (bli.),Slovinsko,"KRKA d.d., Novo mesto",107996.080,297846.23
4,144,A02BC02,Pantoprazol,C43791,Pantoprazol Sandoz 20 mg gastrorezistentné tab...,Slovinsko,Sandoz Pharmaceuticals d.d.,73014.340,105977.50


,index,ATC7_KOD,ATC7_popis,LIEK_KOD,LIEK_popis,STAT_popis,VYROBCA_popis,Počet_balení,Úhrada_PAC
0,7819,R01AA09,Tramazolín,C26797,MUCONASAL PLUS sol neb 1x10 ml,Slovensko,sanofi-aventis Slovakia s.r.o.,1135312.849,5548815.77


# Grafy pre jednotlive ucinne latky

# 1. Technecium

In [14]:
technecium = top20tab["V09FX01"]

for i,popis in enumerate(technecium['LIEK_popis']):
    if 'Drytec gen' in popis:
        technecium.iloc[i,4] = 'Drytec gen rnk'
    elif "Ultra-TechneKow" in popis:
        technecium.iloc[i,4] = 'Ultra-TechneKow FM (99mTC-generátor)'

technecium = technecium.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(technecium)
print()
print("Typ vydaju:")
item_counts = table.query('ATC7_KOD == "V09FX01"')["TYP_VYDAJ_POP"].value_counts(normalize=True)
print(item_counts)



,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,Drytec gen rnk,Holandsko,GE Healthcare B.V.,57267,8626479.51,0.0
1,Ultra-TechneKow FM (99mTC-generátor),Holandsko,Curium Netherlands B.V.,38158,992000.00,0.0



Typ vydaju:
výdaj z nemocničnej lekárne pre vlastnú nemocnicu    0.692308
úhrada z verejnej poisťovne                          0.307692
Name: TYP_VYDAJ_POP, dtype: float64


In [15]:
fig = px.bar(technecium, x='LIEK_popis', y='Počet_balení', color='VYROBCA_popis', title='Technecium')
fig.show()

 Na vizualizaciu ucinnej latky technecium nam bolo najvyhodnejsie pouzit barplot. 
 Nakolko sa jedna o diagnosticke radiofarmaka, podla vypisu Series vidime ze tieto lieky boli poskytovane nemocnicou alebo ich
 pacientom hradila zdravotna poistovna. V oboch pripadoch pacient neplatil nic, a tak udaje o cene nemame ako zakomponovat. Obe radiofarmaka su vyrabane Holandskymi vyrobcami, preto v grafe nie su rozlisene na zaklade krajiny ale vyrobcu.

# 2. Elektrolyty:

In [56]:
elektrolyty = top20tab["B05BB01"]

fig = px.bar(elektrolyty.groupby(['VYROBCA_popis','STAT_popis']).sum().reset_index().query('Počet_balení > 500000'), x='VYROBCA_popis', y='Počet_balení',title='Elektrolyty podla vyrobcov nad 500 000', color='STAT_popis')
fig.show()

In [58]:
elektrolyty_staty = elektrolyty.groupby('STAT_popis').sum().reset_index()
fig = px.pie(elektrolyty_staty, names='STAT_popis', values='Počet_balení', title='Elektrolyty podla krajiny vyrobcu (vsetky)')
fig.show()

Vidime, ze najviac (61.8%) pouzivame ceskych elektorlytov, co je vyse 3 000 000. Slovenske elektrolyty su az na poslednom mieste iba so 4.83% co je priblizne 250 000.

# 3. Paracetamol (30 najpredavanejsich)

In [17]:
paracetamol = top20tab["N02BE01"].sort_values(by='Počet_balení', ascending=False).head(30)

for i,popis in enumerate(paracetamol['LIEK_popis']):
    if 'MEDIPYRIN 500 tbl' in popis:
        paracetamol.iloc[i,4] = 'MEDIPYRIN tbl'
    elif 'PANADOL BABY (+odmerka)' in popis:
        paracetamol.iloc[i,4] = 'PANADOL BABY (+odmerka)'
    elif 'PANADOL BABY (+striekačka)' in popis:
        paracetamol.iloc[i,4] = 'PANADOL BABY (+striekačka)'
    elif 'PANADOL BABY' in popis:
        paracetamol.iloc[i,4] = 'PANADOL BABY'
    elif 'PANADOL JUNIOR' in popis:
        paracetamol.iloc[i,4] = 'PANADOL JUNIOR'
    elif 'PANADOL PRE DETI' in popis:
        paracetamol.iloc[i,4] = 'PANADOL PRE DETI'
    elif 'PANADOL tbl' in popis:
        paracetamol.iloc[i,4] = 'PANADOL tbl'
    elif 'PARALEN' in popis and 'sus' in popis:
        paracetamol.iloc[i,4] = 'PARALEN sus'
    elif 'PARALEN' in popis and 'tbl' in popis:
        paracetamol.iloc[i,4] = 'PARALEN tbl'
    elif 'PARALEN' in popis and 'sup' in popis:
        paracetamol.iloc[i,4] = 'PARALEN sup'
    elif 'PARALEN' in popis:
        paracetamol.iloc[i,4] = 'PARALEN nápoj'
    elif 'Panadol Novum' in popis:
        paracetamol.iloc[i,4] = 'Panadol Novum tbl'
    elif 'Panadol Paracetamolum' in popis:
        paracetamol.iloc[i,4] = 'Panadol Paracetamolum tbl'
    elif 'Paracetamol B. Braun' in popis:
        paracetamol.iloc[i,4] = 'Paracetamol B. Braun'
    elif 'Paracetamol Kabi' in popis:
        paracetamol.iloc[i,4] = 'Paracetamol Kabi'
    elif 'Paracetamol Kappler' in popis:
        paracetamol.iloc[i,4] = 'Paracetamol Kappler'
    elif 'Paracetamol Vitabalans' in popis:
        paracetamol.iloc[i,4] = 'Paracetamol Vitabalans'

paracetamol = paracetamol.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
print("Typ vydaju:")
item_counts = table.query('ATC7_KOD == "N02BE01"')["TYP_VYDAJ_POP"].value_counts(normalize=True)
print(item_counts)

display(paracetamol.sort_values(by='LIEK_popis'))


Typ vydaju:
predané z verejnej lekárne bez receptu občanom (voľnopredajné lieky)    0.304054
výdaj z verejnej lekárne  na recept bez úhrady z verejného poistenia    0.209459
výdaj z verejnej lekárne na faktúru do neštátnych ambulancií            0.162162
výdaj z nemocničnej lekárne pre vlastnú nemocnicu                       0.162162
výdaj z verejnej lekárne na žiadanku nemocniciam                        0.141892
výdaj z nemocničnej lekárne pre zmluvnú nemocnicu                       0.013514
úhrada z verejnej poisťovne                                             0.006757
Name: TYP_VYDAJ_POP, dtype: float64


,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,MEDIPYRIN tbl,Česko,Glenmark Pharmaceuticals s.r.o.,18807,21920.890,36758.59
1,PANADOL BABY,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,6283,50394.000,100522.52
2,PANADOL BABY (+odmerka),Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,6305,839.000,2862.69
3,PANADOL BABY (+striekačka),Francúzsko,SmithKline Beecham Liquides Industrie,6300,285.000,1012.17
4,PANADOL JUNIOR,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,6285,30460.500,77105.53
5,PANADOL PRE DETI,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,12579,37895.000,140531.82
6,PANADOL tbl,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,12580,553.000,942.77
7,PARALEN nápoj,Slovensko,sanofi-aventis Slovakia s.r.o.,6271,482.170,1673.49
8,PARALEN sup,Slovensko,sanofi-aventis Slovakia s.r.o.,12565,161406.210,153352.77
9,PARALEN sus,Slovensko,sanofi-aventis Slovakia s.r.o.,6292,74950.000,270530.14


In [60]:
paracetamol_all = top20tab["N02BE01"].sort_values(by='Počet_balení', ascending=False)
for i,popis in enumerate(paracetamol_all['LIEK_popis']):
    if 'PANADOL' in popis:
        paracetamol_all.iloc[i,4] = 'PANADOL'
    elif 'Panadol' in popis:
        paracetamol_all.iloc[i,4] = 'PANADOL'
    elif 'PARALEN' in popis:
        paracetamol_all.iloc[i,4] = 'PARALEN'
    elif 'Paralen' in popis:
        paracetamol_all.iloc[i,4] = 'PARALEN'
    else:
        paracetamol_all.iloc[i,4] = 'INÉ'
paracetamol_all = paracetamol_all.groupby(['LIEK_popis']).sum().reset_index()
display(paracetamol_all)
fig = px.pie(paracetamol_all, values='Počet_balení', names='LIEK_popis', title='PARACETAMOL (vsetky)')
fig.show()

,LIEK_popis,index,Počet_balení,Úhrada_PAC
0,INÉ,94046,63990.020,56460.27
1,PANADOL,163554,312880.565,677698.50
2,PARALEN,50267,2920149.522,4615756.44


Preco je paralen takto "popularny"? Monzno ze ho ludia viac poznaju, je to prvy liek s paracetamolom co ich napadne, mozno niektorym pride ucinnejsi, ale aj ked sa pozrieme na tabulku liekov s paracetamolom, vidime, ze panadol je zvycajne pre deti.

In [65]:
fig = px.bar(paracetamol,x='LIEK_popis', y='Počet_balení',color='STAT_popis', title='Paracetamol - pocet baleni (top 30)')
fig.update_xaxes(tickangle=45)
fig.show()

Z grafu vidime, ze vsetky lieky prekonava Paralen v tabletach. Tkisto si mozeme vsimnut ze je vyrabany na Slovensku. Cesko ma siroku ponuku Panadolov avsak tieto tak popularne nie su. Panadoly su zvycajne vo variantoch pre deti.

# 4. Ibuprofen

In [20]:
ibuprofen = top20tab["M01AE01"].sort_values(by='Počet_balení', ascending=False).head(30)

for i,popis in enumerate(ibuprofen['LIEK_popis']):
    if 'IBALGIN 400' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN 400'
    elif 'Ibalgin 400' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN 400'
    elif 'BRUFEN 400' in popis:
        ibuprofen.iloc[i,4] = 'BRUFEN 400'
    elif 'Brufen 400' in popis:
        ibuprofen.iloc[i,4] = 'BRUFEN 400'
    elif 'BRUFEN sirup' in popis:
        ibuprofen.iloc[i,4] = 'BRUFEN sirup'
    elif 'BRUFEN retard 800' in popis:
        ibuprofen.iloc[i,4] = 'BRUFEN retard 800'
    elif 'BRUFEN 600' in popis:
        ibuprofen.iloc[i,4] = 'BRUFEN 600'
    elif 'MIG-400' in popis:
        ibuprofen.iloc[i,4] = 'MIG-400'
    elif 'NUROFEN pre deti' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN pre deti'
    elif 'NUROFEN PRE DETI' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN pre deti'
    elif 'Nurofen pre deti' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN pre deti'
    elif 'NUROFEN PRE DETI' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN pre deti'
    elif 'IBALGIN RAPIDCAPS' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN RAPIDCAPS'
    elif 'IBALGIN 200' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN 200'
    elif 'IBALGIN 600' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN 600'
    elif 'IBALGIN FAST' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN FAST'
    elif 'NUROFEN Liquid Capsules' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN Liquid Capsules'
    elif 'IBUPROFEN 400' in popis:
        ibuprofen.iloc[i,4] = 'IBUPROFEN 400'
    elif 'IBALGIN SUS' in popis:
        ibuprofen.iloc[i,4] = 'IBALGIN SUS'
    elif 'MIG' in popis:
        ibuprofen.iloc[i,4] = 'MIG Junior'
    elif 'NUROFEN FORTE' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN FORTE'
    elif 'NUROFEN tbl' in popis:
        ibuprofen.iloc[i,4] = 'NUROFEN tbl'

ibuprofen = ibuprofen.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(ibuprofen)

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,BRUFEN 400,Francúzsko,Mylan Laboratories SAS,11226,260398.011,1000652.93
1,BRUFEN 600,Francúzsko,Mylan Laboratories SAS,5625,28425.100,88020.41
2,IBALGIN 200,Slovensko,sanofi-aventis Slovakia s.r.o.,5603,115937.580,274563.27
3,IBALGIN 400,Slovensko,sanofi-aventis Slovakia s.r.o.,22455,925557.576,3503655.19
4,IBALGIN 600,Slovensko,sanofi-aventis Slovakia s.r.o.,5629,36402.670,64870.46
5,IBALGIN FAST,Slovensko,sanofi-aventis Slovakia s.r.o.,5641,32353.100,108560.61
6,IBALGIN RAPIDCAPS,Slovensko,sanofi-aventis Slovakia s.r.o.,11222,195327.956,899915.02
7,IBALGIN SUS,Slovensko,sanofi-aventis Slovakia s.r.o.,5634,16038.000,52677.24
8,IBUPROFEN 400,Nemecko,STADA Arzneimittel AG,11253,87787.140,228675.29
9,MIG Junior,Nemecko,Berlin-Chemie AG,5571,12276.300,34815.97


In [66]:
fig = px.bar(ibuprofen, x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Ibuprofen - pocet baleni (top 30)')
fig.update_xaxes(tickangle=45)
fig.show()

In [70]:
ibuprofen_all = top20tab["M01AE01"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(ibuprofen_all['LIEK_popis']):
    if 'IBALGIN' in popis:
        ibuprofen_all.iloc[i,4] = 'IBALGIN'
    elif 'Ibalgin' in popis:
        ibuprofen_all.iloc[i,4] = 'IBALGIN'
    elif 'BRUFEN' in popis:
        ibuprofen_all.iloc[i,4] = 'BRUFEN'
    elif 'Brufen' in popis:
        ibuprofen_all.iloc[i,4] = 'BRUFEN'
    elif 'NUROFEN' in popis:
        ibuprofen_all.iloc[i,4] = 'NUROFEN'
    elif 'Nurofen' in popis:
        ibuprofen_all.iloc[i,4] = 'NUROFEN'
    elif 'MIG' in popis:
        ibuprofen_all.iloc[i,4] = 'MIG'
    else:
        ibuprofen_all.iloc[i,4] = 'INÉ'
ibuprofen_all = ibuprofen_all.groupby(['LIEK_popis','STAT_popis']).sum().reset_index()
display(ibuprofen_all)
fig = px.pie(ibuprofen_all, values='Počet_balení', names='LIEK_popis', title='IBUPROFEN')
fig.show()
# Medzi ine su zaradene lieky s poctom baleni pod 100 000

,LIEK_popis,STAT_popis,index,Počet_balení,Úhrada_PAC
0,BRUFEN,Francúzsko,67173,312661.631,1192883.73
1,IBALGIN,Slovensko,106811,1324939.472,4917048.03
2,INÉ,Fínsko,55935,16234.534,35680.04
3,INÉ,Holandsko,5586,5.000,0.20
4,INÉ,Nemecko,44889,100127.660,272368.37
5,INÉ,Rakúsko,5638,288.000,699.84
6,INÉ,Spojené kráľovstvo,16719,35.000,156.93
7,INÉ,ĺrsko,16908,896.900,4148.80
8,MIG,Nemecko,33625,247223.123,793578.27
9,NUROFEN,Francúzsko,5652,6.000,30.35


In [71]:
fig = px.bar(ibuprofen_all, y='Počet_balení', x='LIEK_popis', title='IBUPROFEN', color='STAT_popis')
fig.show()

Z grafu vidime, ze najpredavanejsi liek s ibuprofenom je slovensky Ibalgin 400. Z ceskych liekov bol najpredavanejsi Nurofen pre deti. To, ze bol druhy najpredavanejsi je mozno sposobene tym ze je prave pre deti. 

# 5. Bisoprolol

In [23]:
bisoprolol = top20tab["C07AB07"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(bisoprolol['LIEK_popis']):
    if 'BISOBELA' in popis:
        bisoprolol.iloc[i,4] = 'BISOBELA'
    elif 'BISOCARD' in popis:
        bisoprolol.iloc[i,4] = 'BISOCARD'
    elif 'BISOGAMMA' in popis:
        bisoprolol.iloc[i,4] = 'BISOGAMMA'
    elif 'BISOMERCK' in popis:
        bisoprolol.iloc[i,4] = 'BISOMERCK'
    elif 'BISOPROLOL-ratiopharm' in popis:
        bisoprolol.iloc[i,4] = 'BISOPROLOL-ratiopharm'
    elif 'BIVAXOL' in popis:
        bisoprolol.iloc[i,4] = 'BIVAXOL'
    elif 'Bisoprolol Pfizer' in popis:
        bisoprolol.iloc[i,4] = 'Bisoprolol Pfizer'
    elif 'Bisoprolol Sandoz' in popis:
        bisoprolol.iloc[i,4] = 'Bisoprolol Sandoz'
    elif 'Bisprotin ' in popis:
        bisoprolol.iloc[i,4] = 'Bisprotin '
    elif 'CONCOR COR' in popis or 'Concor COR' in popis:
        bisoprolol.iloc[i,4] = 'CONCOR COR'
    elif 'CONCOR' in popis:
        bisoprolol.iloc[i,4] = 'CONCOR'
    elif 'CORONAL' in popis:
        bisoprolol.iloc[i,4] = 'CORONAL'
    elif 'RIVOCOR' in popis:
        bisoprolol.iloc[i,4] = 'RIVOCOR'
    elif 'Sobycor' in popis:
        bisoprolol.iloc[i,4] = 'Sobycor'

bisoprolol = bisoprolol.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(bisoprolol)

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,BISOBELA,Slovensko,"BELUPO, s.r.o.",5957,7197.330,16626.62
1,BISOCARD,Česko,PharmaSwiss Česká republika s.r.o.,7954,192394.600,151460.50
2,BISOGAMMA,Nemecko,Wörwag Pharma GmbH & Co.KG,1979,2.000,6.34
3,BISOMERCK,Slovensko,Merck spol. s.r.o.,4009,122739.500,174809.80
4,BISOPROLOL-ratiopharm,Nemecko,ratiopharm GmbH,3949,390962.830,86444.66
5,BIVAXOL,Česko,IVAX Pharmaceuticals s.r.o.,1999,2.000,3.20
6,Bisoprolol Pfizer,Spojené kráľovstvo,Pfizer Europe MA EEIG,3933,119.000,16.40
7,Bisoprolol Sandoz,Slovinsko,Sandoz Pharmaceuticals d.d.,3939,10493.660,2229.01
8,Bisprotin,Fínsko,Vitabalans Oy,9920,1965.900,1922.87
9,CONCOR,Slovensko,Merck spol. s.r.o.,8006,920947.070,2677920.45


In [24]:
fig = px.bar(bisoprolol.query('Počet_balení > 90000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Bisoprolol - balenia (nad 90 000)')
fig.update_xaxes(tickangle=45)
fig.show()

In [72]:
bisoprolol_krajina = bisoprolol.groupby('STAT_popis').sum().reset_index()
for i,popis in enumerate(bisoprolol_krajina['STAT_popis']):
    if 'Fínsko' in popis:
        bisoprolol_krajina.iloc[i,0] = 'iné'
    elif 'Spojené kráľovstvo' in popis:
        bisoprolol_krajina.iloc[i,0] = 'iné'
bisoprolol_krajina = bisoprolol_krajina.groupby('STAT_popis').sum().reset_index()
fig = px.pie(bisoprolol_krajina, values='Počet_balení', names='STAT_popis', title='BISOPROLOL')
fig.show()

Z grafu vidime, ze najpouzivanejsi betablokator s ucinnou latkou bisoprolol je slovensky Concor. Narozdiel od predchadzajucich grafov, vidime, ze aj ine lieky sa vo vyraznej miere pouzivaju, napr Coronal alebo nemecky Bisprolol-ratiopharm. Jednoznacne vedu slovenske lieky.

# 6. Amlodipin

In [26]:

amlodipin = top20tab["C08CA01"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(amlodipin['LIEK_popis']):
    if 'ACCEL' in popis:
        amlodipin.iloc[i,4] = 'ACCEL'
    elif 'AGEN' in popis:
        amlodipin.iloc[i,4] = 'AGEN'
    elif 'AMLODIPIN HEXAL' in popis:
        amlodipin.iloc[i,4] = 'AMLODIPIN HEXAL'
    elif 'AMLODIPIN-RATIOPHARM' in popis:
        amlodipin.iloc[i,4] = 'AMLODIPIN-RATIOPHARM'
    elif 'AMLOPIN' in popis:
        amlodipin.iloc[i,4] = 'AMLOPIN'
    elif 'Amlodigamma' in popis:
        amlodipin.iloc[i,4] = 'Amlodigamma'
    elif 'Amlodipin Actavis' in popis:
        amlodipin.iloc[i,4] = 'Amlodipin Actavis'
    elif 'Amlodipin Stichting' in popis:
        amlodipin.iloc[i,4] = 'Amlodipin Stichting'
    elif 'Amlodipin Vitabalans' in popis:
        amlodipin.iloc[i,4] = '	Amlodipin Vitabalans'
    elif 'CARDILOPIN' in popis:
        amlodipin.iloc[i,4] = 'CARDILOPIN'
    elif 'NORVASC' in popis:
        amlodipin.iloc[i,4] = 'NORVASC'
    elif 'TENOX' in popis:
        amlodipin.iloc[i,4] = 'TENOX'

amlodipin = amlodipin.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(amlodipin)

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,\tAmlodipin Vitabalans,Fínsko,Vitabalans Oy,8326,8133.700,8013.23,0.985189
1,ACCEL,Cyprus,Medochemie Ltd.,4199,39635.000,36396.58,0.918294
2,AGEN,Česko,"Zentiva, k.s.",8352,1042447.713,514840.56,0.493877
3,AMLODIPIN HEXAL,Dánsko,HEXAL A/S,2106,2.000,13.46,6.730000
4,AMLODIPIN-RATIOPHARM,Nemecko,ratiopharm GmbH,4179,402849.470,79580.14,0.197543
5,AMLOPIN,Slovinsko,LEK Pharmaceuticals d.d.,4205,13.000,16.80,1.292308
6,Amlodigamma,Nemecko,Artesan Pharma GmbH & Co. KG,2093,2.000,30.12,15.060000
7,Amlodipin Actavis,Island,Actavis Group PTC ehf,4187,16.000,13.50,0.843750
8,Amlodipin Stichting,Slovinsko,Sandoz Pharmaceuticals d.d.,6285,415811.500,133785.38,0.321745
9,CARDILOPIN,Maďarsko,EGIS Pharmaceuticals PLC,4175,62800.500,79712.18,1.269292


In [74]:

fig = px.bar(amlodipin.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Amlodipin nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()
display(amlodipin)

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,\tAmlodipin Vitabalans,Fínsko,Vitabalans Oy,8326,8133.700,8013.23,0.985189
1,ACCEL,Cyprus,Medochemie Ltd.,4199,39635.000,36396.58,0.918294
2,AGEN,Česko,"Zentiva, k.s.",8352,1042447.713,514840.56,0.493877
3,AMLODIPIN HEXAL,Dánsko,HEXAL A/S,2106,2.000,13.46,6.730000
4,AMLODIPIN-RATIOPHARM,Nemecko,ratiopharm GmbH,4179,402849.470,79580.14,0.197543
5,AMLOPIN,Slovinsko,LEK Pharmaceuticals d.d.,4205,13.000,16.80,1.292308
6,Amlodigamma,Nemecko,Artesan Pharma GmbH & Co. KG,2093,2.000,30.12,15.060000
7,Amlodipin Actavis,Island,Actavis Group PTC ehf,4187,16.000,13.50,0.843750
8,Amlodipin Stichting,Slovinsko,Sandoz Pharmaceuticals d.d.,6285,415811.500,133785.38,0.321745
9,CARDILOPIN,Maďarsko,EGIS Pharmaceuticals PLC,4175,62800.500,79712.18,1.269292


Mozeme si vsimnut ze najpouzivanejsie blokator vapnikovych kamenov s ucinnou latkou amlodipin je cesky Agen. Dalsimi ale menej pouzivanymi su slovinsky Amlodipin Stichting a nemecky Amlodipin-ratiopharm. Podla tabulky vidime, ze Slovensko taketo blokatory vapnikovych kamenov nevyraba.

# 7. Alprazolam

In [28]:

alprazolam = top20tab["N05BA12"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(alprazolam['LIEK_popis']):
    if 'Alprazolam Orion' in popis:
        alprazolam.iloc[i,4] = 'Alprazolam Orion'
    elif 'FRONTIN' in popis:
        alprazolam.iloc[i,4] = 'FRONTIN'
    elif 'HELEX' in popis:
        alprazolam.iloc[i,4] = 'HELEX'
    elif 'Helex' in popis:
        alprazolam.iloc[i,4] = 'HELEX'
    elif 'XANAX' in popis:
        alprazolam.iloc[i,4] = 'XANAX'
    elif 'NEUROL SR' in popis:
        alprazolam.iloc[i,4] = 'NEUROL SR'
    elif 'NEUROL' in popis:
        alprazolam.iloc[i,4] = 'NEUROL'

alprazolam = alprazolam.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(alprazolam)

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC,Cena úhrada_PAC/bal
0,Alprazolam Orion,Fínsko,"Orion Corporation, Orion Pharma",7183,2.00,5.14,2.570000
1,FRONTIN,Maďarsko,EGIS Pharmaceuticals PLC,35945,537957.20,731856.63,1.360437
2,HELEX,Slovinsko,"KRKA d.d., Novo mesto",43134,65145.00,65413.51,1.004122
3,NEUROL,Česko,"Zentiva, k.s.",21597,1056309.50,1395763.59,1.321359
4,NEUROL SR,Česko,Zentiva a.s.,7196,2.00,6.26,3.130000
5,XANAX,Belgicko,Pfizer Europe MA EEIG,43202,385706.03,647377.73,1.678423


In [76]:
fig = px.bar(alprazolam.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Alprazolam nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()


 Najpuzivanejsim psycholeptikom s ucinnou latkou alprazolam je cesky Neurol. Priblizne o polovicu menej sa predavajuci madarsky Frontin a potom belgickz Xanax. Mozeme si vsimnut ze narozdiel od predchadzajuch grafov, ucinnu latku alprazolam pouziva menej liekov. Slovensko opat taketo psycholeptika nevyraba

# 8. Diklofenak

In [77]:
diklofenak = top20tab["M01AB05"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(diklofenak['LIEK_popis']):
    if 'AKIS Plus' in popis:
        diklofenak.iloc[i,4] = 'AKIS Plus inj'
    elif 'AKIS' in popis:
        diklofenak.iloc[i,4] = 'AKIS inj'
    elif 'ALMIRAL inj' in popis:
        diklofenak.iloc[i,4] = 'ALMIRAL inj'
    elif 'ALMIRAL ' in popis:
        diklofenak.iloc[i,4] = 'ALMIRAL tbl'
    elif 'DICLAC RETARD tbl' in popis or 'Diclac retard' in popis:
        diklofenak.iloc[i,4] = 'DICLAC RETARD tbl'
    elif 'DICLOBENE' in popis and 'RETARD' in popis:
        diklofenak.iloc[i,4] = 'DICLOBENE RETARD tbl'
    elif 'DICLOBENE' in popis:
        diklofenak.iloc[i,4] = 'DICLOBENE tbl'
    elif 'DICLOFENAC AL RETARD' in popis:
        diklofenak.iloc[i,4] = 'DICLOFENAC AL RETARD tbl'
    elif 'DICLOFENAC AL' in popis:
        diklofenak.iloc[i,4] = 'DICLOFENAC AL tbl'
    elif 'DICLOFENAC DUO PHARMAVIT' in popis:
        diklofenak.iloc[i,4] = 'DICLOFENAC DUO PHARMAVIT cps'
    elif 'DICLOREUM tbl' in popis:
        diklofenak.iloc[i,4] = 'DICLOREUM tbl'
    elif 'DICLOREUM' in popis:
        diklofenak.iloc[i,4] = 'DICLOREUM sup'
    elif 'Dicuno' in popis:
        diklofenak.iloc[i,4] = 'Dicuno tbl'
    elif 'FLECTOR EP RAPID' in popis:
        diklofenak.iloc[i,4] = 'FLECTOR EP RAPID'
    elif 'VERAL' in popis and 'RETARD' in popis:
        diklofenak.iloc[i,4] = 'VERAL RETARD tbl'
    elif 'VERAL' in popis and 'tbl' in popis:
        diklofenak.iloc[i,4] = 'VERAL tbl'
    elif 'VERAL' in popis and 'sup' in popis:
        diklofenak.iloc[i,4] = 'VERAL sup'
    elif 'VERAL' in popis and 'sol' in popis:
        diklofenak.iloc[i,4] = 'VERAL sol inj'
    elif 'VERAL' in popis and 'gel' in popis:
        diklofenak.iloc[i,4] = 'VERAL gel'
    elif 'VOLTAREN 100 sup' in popis:
        diklofenak.iloc[i,4] = 'VOLTAREN sup'
    elif 'VOLTAREN 50 tbl' in popis:
        diklofenak.iloc[i,4] = 'VOLTAREN tbl'
    elif 'VOLTAREN RAPID' in popis:
        diklofenak.iloc[i,4] = 'VOLTAREN RAPID tbl'
    elif 'VOLTAREN' in popis and 'sol' in popis:
        diklofenak.iloc[i,4] = 'VOLTAREN sol ifc'
    elif 'Voltaren ActiGo Extra' in popis:
        diklofenak.iloc[i,4] = 'Voltaren ActiGo Extra tbl'
    elif 'Voltaren Actigo Extra' in popis:
        diklofenak.iloc[i,4] = 'Voltaren ActiGo Extra tbl'
    elif 'Voltaren Rapid' in popis:
        diklofenak.iloc[i,4] = 'Voltaren Rapid cps'
    elif 'VOLTAREN RETARD' in popis:
        diklofenak.iloc[i,4] = 'VOLTAREN RETARD tbl'
    
diklofenak = diklofenak.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()
display(diklofenak)

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,AKIS Plus inj,Slovensko,IBSA Slovakia s.r.o.,21978,6095.903,51936.89
1,AKIS inj,Slovensko,IBSA Slovakia s.r.o.,21954,270.630,3414.97
2,ALMIRAL inj,Cyprus,Medochemie Ltd.,16503,167768.830,104943.58
3,ALMIRAL tbl,Cyprus,Medochemie Ltd.,5499,28693.990,23225.76
4,DICLAC RETARD tbl,Slovinsko,Sandoz Pharmaceuticals d.d.,10966,11883.500,15556.74
5,DICLOBENE RETARD tbl,Holandsko,TEV-NL-4?,16407,57957.000,29930.63
6,DICLOBENE tbl,Holandsko,TEV-NL-4?,21862,46020.650,25987.73
7,DICLOFENAC AL RETARD tbl,Nemecko,ALIUD PHARMA GmbH,11011,23.000,104.92
8,DICLOFENAC AL tbl,Nemecko,Aliud Pharma GmbH & Co. KG,11013,4.000,14.11
9,DICLOFENAC DUO PHARMAVIT cps,Česko,PharmaSwiss Česká republika s.r.o.,5510,170025.010,469116.41


In [79]:
fig = px.bar(diklofenak.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Diklofenak nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()

Z grafu vidime, ze najpredavanejsi je slovensky Flector Ep rapid. V tejto oblasti su bohato zastupene slovenske a ceske lieky.

In [81]:
fig = px.pie(diklofenak.groupby('STAT_popis').sum().reset_index(), names='STAT_popis', values='Počet_balení', title = 'Diklofenak podla krajiny vyrobcu (vsetky)')
fig.show()

# 9. Metamizol, sodna sol

In [82]:

metamizol = top20tab["N02BB02"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(metamizol['LIEK_popis']):
    if 'METAMISTAD' in popis:
        metamizol.iloc[i,4] = 'METAMISTAD tbl'
    elif 'Metamilan' in popis:
        metamizol.iloc[i,4] = 'Metamilan tbl'
    elif 'Metamizol Stada' in popis:
        metamizol.iloc[i,4] = 'Metamizol Stada'
    elif 'Metamizol Teva' in popis:
        metamizol.iloc[i,4] = 'Metamizol Teva'
    elif 'Metamizole Kalceks' in popis:
        metamizol.iloc[i,4] = 'Metamizole Kalceks'
    elif 'NOVALGIN' in popis and 'tbl' in popis:
        metamizol.iloc[i,4] = 'NOVALGIN tbl'
    elif 'NOVALGIN' in popis:
        metamizol.iloc[i,4] = 'NOVALGIN sol inj'
    

metamizol = metamizol.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

display(metamizol.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,METAMISTAD tbl,Nemecko,STADA Arzneimittel AG,12509,183829.997,74327.59
1,Metamilan tbl,Rakúsko,G.L.Pharma GmbH,6249,32683.300,8515.62
2,Metamizol Stada,Nemecko,STADA Arzneimittel AG,12513,112570.900,96246.06
3,Metamizol Teva,Holandsko,TEV-NL-4?,6244,1.000,0.10
4,Metamizole Kalceks,Lotyšsko,AS KALCEKS,12501,56516.500,1313.88
5,NOVALGIN sol inj,Nemecko,Aventis Pharma Deutschland GmbH,6253,3.000,0.00
6,NOVALGIN sol inj,Slovensko,sanofi-aventis Slovakia s.r.o.,12491,218420.700,162095.26
7,NOVALGIN tbl,Slovensko,sanofi-aventis Slovakia s.r.o.,12495,1342223.310,1574153.23
8,NOVALGIN tbl,Taliansko,Aventis Pharma S.p.A.,6252,2.000,4.24


In [84]:
fig = px.bar(metamizol.query('Počet_balení > 100000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Metamizol, sodna sol nad 100 000')
fig.update_xaxes(tickangle=45)
fig.show()

Z grafu vidime, ze sa najviac predava slovensky Novalgin v tabletach. 

# 10. Metoprolol

In [99]:
metoprolol = top20tab["C07AB02"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(metoprolol['LIEK_popis']):
    if 'BETALOC' in popis:
        metoprolol.iloc[i,4] = 'BETALOC'
    elif 'CORVITOL' in popis:
        metoprolol.iloc[i,4] = 'CORVITOL'
    elif 'EGILOK' in popis:
        metoprolol.iloc[i,4] = 'EGILOK'
    elif 'Egilok' in popis:
        metoprolol.iloc[i,4] = 'EGILOK'
    elif 'Metoprobene' in popis:
        metoprolol.iloc[i,4] = 'METOPROBENE'
    elif 'Metoprolol Polpharma' in popis:
        metoprolol.iloc[i,4] = 'Metoprolol Polpharma'
    elif 'Metoprolol ratiopharm' in popis:
        metoprolol.iloc[i,4] = 'Metoprolol ratiopharm'
    elif 'VASOCARDIN' in popis:
        metoprolol.iloc[i,4] = 'VASOCARDIN'

metoprolol = metoprolol.groupby(['LIEK_popis', 'STAT_popis']).sum().reset_index()
display(metoprolol.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,index,Počet_balení,Úhrada_PAC
0,BETALOC,Taliansko,7783,917306.500,2991813.86
1,CORVITOL,Nemecko,1958,97308.000,13077.12
2,EGILOK,Maďarsko,21348,157033.400,136782.42
3,METOPROBENE,Nemecko,1947,1.000,6.72
4,Metoprolol Polpharma,Poľsko,1948,3.000,23.22
5,Metoprolol ratiopharm,Nemecko,7818,368367.099,134140.37
6,VASOCARDIN,Slovensko,7781,333583.520,27500.64


In [101]:
fig = px.bar(metoprolol.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Metoprolol nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()

Preco nie je slovensky najpredavanejsi?

# 11. Kyselina acetylsalicylova

In [85]:

k_acetylsalicylova = top20tab["B01AC06"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(k_acetylsalicylova['LIEK_popis']):
    if 'ANOPYRIN' in popis:
        k_acetylsalicylova.iloc[i,4] = 'ANOPYRIN'
    elif 'ASPIRIN PROTECT' in popis:
        k_acetylsalicylova.iloc[i,4] = 'ASPIRIN PROTECT'
    elif 'CARDIORAL' in popis:
        k_acetylsalicylova.iloc[i,4] = 'CARDIORAL'
    elif 'Gerasa' in popis:
        k_acetylsalicylova.iloc[i,4] = 'Gerasa'
    elif 'Kyselina acetylsalicylová Xantis' in popis:
        k_acetylsalicylova.iloc[i,4] = 'Kyselina acetylsalicylová Xantis'
    elif 'Preventax' in popis:
        k_acetylsalicylova.iloc[i,4] = 'Preventax'
    elif 'STADAPYRIN' in popis:
        k_acetylsalicylova.iloc[i,4] = 'STADAPYRIN'
    elif 'Vasopirin' in popis:
        k_acetylsalicylova.iloc[i,4] = 'Vasopirin'
   

k_acetylsalicylova = k_acetylsalicylova.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

display(k_acetylsalicylova.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,ANOPYRIN,Slovensko,ZNT-SK-2?,6040,1089219.322,2058022.70
1,ASPIRIN PROTECT,Slovensko,"Bayer, spol. s.r.o.",3986,329049.400,877523.76
2,CARDIORAL,Slovensko,IBSA Slovakia s.r.o.,1015,4.000,186.87
3,Gerasa,Rakúsko,G.L.Pharma GmbH,2027,3931.160,6697.88
4,Kyselina acetylsalicylová Xantis,Cyprus,XANTIS PHARMA LIMITED,2019,10972.540,11696.24
5,Preventax,Holandsko,TEV-NL-4?,1016,76991.310,215135.04
6,STADAPYRIN,Nemecko,STADA Arzneimittel AG,2010,201492.075,554253.19
7,Vasopirin,Česko,PharmaSwiss Česká republika s.r.o.,5025,39774.200,79233.23


In [87]:
fig = px.bar(k_acetylsalicylova.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Kyselina acetylsalicylova nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
Najpredavanejsim liekom s kyselinou acetylsalicylovou je slovensky Anopyrin. Najviac sa predavaju slovenske lieky, ktorych je na trhu najviac.

# 12. Atorvastatin (top 55)

In [88]:
atorvastatin = top20tab["C10AA05"].sort_values(by='Počet_balení', ascending=False).head(55)

for i,popis in enumerate(atorvastatin['LIEK_popis']):
    if 'ATORIS' in popis:
        atorvastatin.iloc[i,4] = 'ATORIS'
    if 'Atoris' in popis:
        atorvastatin.iloc[i,4] = 'ATORIS'
    if 'Atorvastatin Actavis' in popis:
        atorvastatin.iloc[i,4] = 'Atorvastatin Actavis'
    if 'SORTIS' in popis:
        atorvastatin.iloc[i,4] = 'SORTIS'
    if 'Torvacard Novum' in popis:
        atorvastatin.iloc[i,4] = 'Torvacard Novum'
    if 'Tulip' in popis:
        atorvastatin.iloc[i,4] = 'Tulip'
    if 'Atorvastatin Mylan' in popis:
        atorvastatin.iloc[i,4] = 'Atorvastatin Mylan'
    if 'Atorvastatin STADA' in popis:
        atorvastatin.iloc[i,4] = 'Atorvastatin STADA'
    if 'Atorvastatin Teva Pharma' in popis:
        atorvastatin.iloc[i,4] = 'Atorvastatin Teva Pharma'
    if 'Atorvastatin Saneca' in popis:
        atorvastatin.iloc[i,4] = 'Atorvastatin Saneca'
    if 'Gletor' in popis:
        atorvastatin.iloc[i,4] = 'Gletor'

atorvastatin = atorvastatin.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

display(atorvastatin.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,ATORIS,Slovinsko,"KRKA d.d., Novo mesto",31395,561480.081,1159087.78
1,Atorvastatin Actavis,Island,Actavis Group PTC ehf,20047,282689.050,229382.68
2,Atorvastatin Mylan,ĺrsko,Mylan Ireland Limited,8457,18970.680,25666.59
3,Atorvastatin STADA,Nemecko,STADA Arzneimittel AG,11574,25251.000,46411.62
4,Atorvastatin Saneca,Cyprus,XANTIS PHARMA LIMITED,11646,7698.654,1390.62
5,Atorvastatin Teva Pharma,Slovensko,TEVA Pharmaceuticals Slovakia s.r.o.,14197,24108.160,9229.01
6,Gletor,Česko,Glenmark Pharmaceuticals s.r.o.,5773,4298.520,7921.31
7,SORTIS,Belgicko,Pfizer Europe MA EEIG,8631,128401.967,180299.01
8,Torvacard Novum,Česko,"Zentiva, k.s.",22781,363483.663,639138.51
9,Tulip,Slovinsko,Sandoz Pharmaceuticals d.d.,22706,152051.083,307103.12


In [90]:
fig = px.bar(atorvastatin.query('Počet_balení > 100000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Atorovastatin nad 100 000')
fig.update_xaxes(tickangle=45)
fig.show()

Najpredavanejsi je slovinsky Atoris, avsak tiez celkom predavanym je aj cesky Torvacard Novum

# 13. Zolpidem

In [105]:

zolpidem = top20tab["N05CF02"].sort_values(by='Počet_balení', ascending=False).head(55)

for i,popis in enumerate(zolpidem['LIEK_popis']):
    if 'ADORMA' in popis:
        zolpidem.iloc[i,4] = 'ADORMA'
    elif 'HYPNOGEN' in popis:
        zolpidem.iloc[i,4] = 'HYPNOGEN'
    elif 'SANVAL' in popis:
        zolpidem.iloc[i,4] = 'SANVAL'
    elif 'STILNOX' in popis:
        zolpidem.iloc[i,4] = 'STILNOX'
    elif 'ZOLPIDEM MERCK' in popis:
        zolpidem.iloc[i,4] = 'ZOLPIDEM MERCK'
    elif 'Zolpidem Orion' in popis:
        zolpidem.iloc[i,4] = 'Zolpidem Orion'
    elif 'Zolpidem Pfizer' in popis:
        zolpidem.iloc[i,4] = 'Zolpidem Pfizer'
    elif 'Zolpidem Saneca' in popis:
        zolpidem.iloc[i,4] = 'Zolpidem Saneca'
    elif 'Zolpidem Teva' in popis:
        zolpidem.iloc[i,4] = 'Zolpidem Teva'
    elif 'Zolpidem Vitabalans' in popis:
        zolpidem.iloc[i,4] = 'Zolpidem Vitabalans'
    elif 'Zolpinox' in popis:
        zolpidem.iloc[i,4] = 'Zolpinox'

zolpidem = zolpidem.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

display(zolpidem.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,ADORMA,Slovinsko,"Alkaloid - INT d.o.o., Ljubljana",7246,1.000,2.00
1,HYPNOGEN,Nemecko,Takeda GmbH,14504,249315.840,407727.12
2,SANVAL,Slovinsko,Sandoz Pharmaceuticals d.d.,14489,242727.100,231796.13
3,STILNOX,Slovensko,sanofi-aventis Slovakia s.r.o.,21756,866042.660,1450471.81
4,ZOLPIDEM MERCK,ĺrsko,Mylan Ireland Limited,7256,97213.240,154214.22
5,Zolpidem Orion,Fínsko,Orion Corporation,14497,9.000,12.45
6,Zolpidem Pfizer,Taliansko,PFIZER Italia S.r.l.,7240,3.000,7.65
7,Zolpidem Saneca,Cyprus,XANTIS PHARMA LIMITED,14505,20968.000,18911.62
8,Zolpidem Teva,Holandsko,Synthon BV,7254,1.500,3.65
9,Zolpidem Vitabalans,Fínsko,Vitabalans Oy,21726,3381.300,6249.51


In [107]:
fig = px.bar(zolpidem.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Zolpidem nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()

# 14. Xylometazolín

In [113]:
xylometazolin = top20tab["R01AA07"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(xylometazolin['LIEK_popis']):
    if 'DR.RENTSCHLER SCHNUPFENSPRAY' in popis:
        xylometazolin.iloc[i,4] = 'DR.RENTSCHLER SCHNUPFENSPRAY'
    elif 'Maresyl' in popis:
        xylometazolin.iloc[i,4] = 'Maresyl'
    elif 'NASENSPRAY' in popis:
        xylometazolin.iloc[i,4] = 'NASENSPRAY'
    elif 'NASENTROPFEN' in popis:
        xylometazolin.iloc[i,4] = 'NASENTROPFEN'
    elif 'OLYNTH HA' in popis:
        xylometazolin.iloc[i,4] = 'OLYNTH HA'
    elif 'OLYNTH' in popis:
        xylometazolin.iloc[i,4] = 'OLYNTH'
    elif 'OTRIVIN' in popis:
        xylometazolin.iloc[i,4] = 'OTRIVIN'
    elif 'Orinox' in popis:
        xylometazolin.iloc[i,4] = 'Orinox'
    elif 'Otrivin Menthol' in popis:
        xylometazolin.iloc[i,4] = 'Otrivin Menthol'
    elif 'Rinoxyl' in popis:
        xylometazolin.iloc[i,4] = 'Rinoxyl'
    elif 'mar rhino' in popis:
        xylometazolin.iloc[i,4] = 'mar rhino'    

xylometazolin = xylometazolin.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

display(xylometazolin.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,DR.RENTSCHLER SCHNUPFENSPRAY,Nemecko,Dr. Rentschler Arzneimittel GmbH & Co.,7797,127.0,909.99
1,Maresyl,Francúzsko,Mylan Laboratories SAS,15583,10395.0,35427.43
2,NASENSPRAY,Nemecko,Aliud Pharma GmbH & Co. KG,7811,4.0,12.95
3,NASENSPRAY,Nemecko,STADA Arzneimittel AG,7813,1.0,8.30
4,NASENTROPFEN,Nemecko,Aliud Pharma GmbH & Co. KG,7805,2.0,9.45
5,OLYNTH,Francúzsko,PFIZER PGM,15605,86.0,349.84
6,OLYNTH,Spojené kráľovstvo,McNeil Products Limited C/O Johnson & Johnson ...,7801,1.0,0.06
7,OLYNTH,ĺrsko,McNeil Healthcare (Ireland) Limited,15618,621781.0,2336426.43
8,OLYNTH HA,ĺrsko,McNeil Healthcare (Ireland) Limited,15599,529046.0,2541167.62
9,OTRIVIN,Česko,GlaxoSmithKline Consumer Healthcare Czech Repu...,31178,95259.5,450124.85


In [115]:
fig = px.bar(xylometazolin.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Xylometazolin nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()

# 15. Rôzne iné krčné antiseptiká

In [124]:
ine_krcne = top20tab["R02AA20"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(ine_krcne['LIEK_popis']):
    if 'Dorithricin pas' in popis:
        ine_krcne.iloc[i,4] = 'Dorithricin pas'
    elif 'Faryngal 5 mg/1 mg lisované' in popis:
        ine_krcne.iloc[i,4] = 'Faryngal pas'
    elif 'Faryngal' in popis:
        ine_krcne.iloc[i,4] = 'Faryngal aerodis'
    elif 'HEXORALETTEN' in popis:
        ine_krcne.iloc[i,4] = 'HEXORALETTEN N pas'
    elif 'NEO-ANGIN' in popis:
        ine_krcne.iloc[i,4] = 'NEO-ANGIN pas'
    elif 'SEPTISAN tbl' in popis:
        ine_krcne.iloc[i,4] = 'SEPTISAN tbl'
    elif 'SEPTOLETE' in popis:
        ine_krcne.iloc[i,4] = 'SEPTOLETE'
    elif 'STOPANGIN' in popis:
        ine_krcne.iloc[i,4] = 'STOPANGIN'
    elif 'STREPSILS' in popis:
        ine_krcne.iloc[i,4] = 'STREPSILS'
    elif 'Septolete' in popis:
        ine_krcne.iloc[i,4] = 'SEPTOLETE'
    elif 'Strepsils' in popis:
        ine_krcne.iloc[i,4] = 'SEPTOLETE'
    elif 'neo-angin' in popis:
        ine_krcne.iloc[i,4] = 'NEO-ANGIN'
      

ine_krcne = ine_krcne.groupby(['LIEK_popis', 'STAT_popis', 'VYROBCA_popis']).sum().reset_index()

display(ine_krcne.sort_values(by='LIEK_popis'))

,LIEK_popis,STAT_popis,VYROBCA_popis,index,Počet_balení,Úhrada_PAC
0,Dorithricin pas,Nemecko,Medice Pharma GmbH & Co. KG,15809,653457.650,4647914.33
1,Faryngal aerodis,Slovinsko,Sandoz Pharmaceuticals d.d.,7929,3846.000,14446.95
2,Faryngal pas,Slovinsko,Sandoz Pharmaceuticals d.d.,15795,11373.370,33328.57
3,HEXORALETTEN N pas,ĺrsko,McNeil Healthcare (Ireland) Limited,7901,3768.500,5789.35
4,NEO-ANGIN,Nemecko,Divapharma GmbH,7907,11357.380,68407.89
5,NEO-ANGIN pas,Nemecko,Divapharma GmbH,31692,105924.680,680358.98
6,SEPTISAN tbl,Slovensko,sanofi-aventis Slovakia s.r.o.,7936,90881.000,301647.58
7,SEPTOLETE,Slovinsko,"KRKA d.d., Novo mesto",63315,109671.600,601482.25
8,SEPTOLETE,Česko,"Reckitt Benckiser (Czech Republic), spol. s r.o.",7919,13398.600,88941.37
9,STOPANGIN,Česko,Teva Czech Industries s.r.o.,15802,63500.500,323868.66


In [126]:
fig = px.bar(ine_krcne.query('Počet_balení > 50000'), x='LIEK_popis', y='Počet_balení', color='STAT_popis', title='Rôzne iné krčné antiseptiká nad 50 000')
fig.update_xaxes(tickangle=45)
fig.show()

In [135]:
strepsils = top20tab["R02AA20"].sort_values(by='Počet_balení', ascending=False)

for i,popis in enumerate(strepsils['LIEK_popis']):
    if 'STREPSILS' not in popis and 'Strepsils' not in popis:
        strepsils.iloc[i,4] = 'ine'
      
strepsils = strepsils.sort_values(by='LIEK_popis').iloc[0:20,:]

for i,popis in enumerate(strepsils['LIEK_popis']):
    if 'CITRON A BYLINY' in popis:
        strepsils.iloc[i,4] = 'CITRON A BYLINY'
    elif 'COOLING MINT' in popis:
        strepsils.iloc[i,4] = 'COOLING MINT'
    elif 'MED A CITRÓN' in popis:
        strepsils.iloc[i,4] = 'MED A CITRÓN'
    elif 'Med a Citrón' in popis:
        strepsils.iloc[i,4] = 'MED A CITRÓN'
    elif 'Menthol a Eucalyptus' in popis:
        strepsils.iloc[i,4] = 'Menthol a Eucalyptus'
    elif 'ORIGINAL' in popis:
        strepsils.iloc[i,4] = 'ORIGINAL'
    elif 'PLUS' in popis:
        strepsils.iloc[i,4] = 'PLUS'
    elif 'POMARANČ S VITAMÍNOM C' in popis:
        strepsils.iloc[i,4] = 'POMARANČ S VITAMÍNOM C'
    elif 'Vitamin C' in popis:
        strepsils.iloc[i,4] = 'Vitamin C'
    elif 'Jahoda bez cukru' in popis:
        strepsils.iloc[i,4] = 'Jahoda bez cukru'
strepsils = strepsils.groupby(['LIEK_popis']).sum().reset_index()

display(strepsils)

,LIEK_popis,index,Počet_balení,Úhrada_PAC
0,CITRON A BYLINY,15845,44667.250,301320.29
1,COOLING MINT,7935,44157.520,280492.45
2,Jahoda bez cukru,7919,13398.600,88941.37
3,MED A CITRÓN,31677,149374.240,945308.40
4,Menthol a Eucalyptus,39560,2433.500,14849.97
5,ORIGINAL,15845,110.500,691.05
6,PLUS,15821,76760.689,566793.42
7,POMARANČ S VITAMÍNOM C,7934,26.500,163.60
8,Vitamin C,15829,59584.480,373220.80


In [139]:
fig = px.bar(strepsils, x='LIEK_popis', y='Počet_balení', title='Ktora prichut Strepsils pastiliek je najpredavanejsia?')
fig.update_xaxes(tickangle=45)
fig.show()

In [140]:
fig = px.pie(strepsils, names='LIEK_popis', values='Počet_balení', title='Ktora prichut Strepsils pastiliek je najpredavanejsia?')
fig.show()